In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import re
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
all_verbs = pd.read_csv('./all_verbs_tree.csv').reset_index(drop = True)
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number,Vowels Quantity
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692,0,ɛ,15,6
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000,0,i,5,5
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636,0,e,3,5
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333,0,ɛ,15,6
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692,0,ɛ,15,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44107,44107,past,subjunctive,pluperfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
44108,44108,past,subjunctive,pluperfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
44109,44109,past,subjunctive,pluperfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ,0.500000,0,n,8,5
44110,44110,past,subjunctive,pluperfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ,0.500000,0,n,8,5


In [3]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [4]:
all_characters = all_verbs['Generated Verb-SR'].sum() #concatinating all characters together

In [5]:
all_characters = sorted(list(set([char for char in all_characters])))
all_characters

['*',
 ':',
 'a',
 'b',
 'd',
 'e',
 'f',
 'g',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'z',
 'ø',
 'ɛ',
 'ɢ',
 'ʃ',
 'ʌ',
 'ʒ',
 'ˈ',
 '͡',
 'χ']

In [6]:
word = 'sdhahhhhhks'
sorted(list(set([char for char in word]))) #list comprehension #set gives the unique valies #turn it into list again, makes it easier to manipulate it later and finally sort it

['a', 'd', 'h', 'k', 's']

In [7]:
char_index = {char: idx+1 for idx, char in enumerate(all_characters) } # input cha, gives you a no #inumerate: iterate a list, returns the element and its index
index_char = {idx+1: char for idx, char in enumerate(all_characters)}
char_index
 # plus one is to start from 1

{'*': 1,
 ':': 2,
 'a': 3,
 'b': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'z': 22,
 'ø': 23,
 'ɛ': 24,
 'ɢ': 25,
 'ʃ': 26,
 'ʌ': 27,
 'ʒ': 28,
 'ˈ': 29,
 '͡': 30,
 'χ': 31}

In [8]:
index_char

{1: '*',
 2: ':',
 3: 'a',
 4: 'b',
 5: 'd',
 6: 'e',
 7: 'f',
 8: 'g',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'r',
 18: 's',
 19: 't',
 20: 'u',
 21: 'v',
 22: 'z',
 23: 'ø',
 24: 'ɛ',
 25: 'ɢ',
 26: 'ʃ',
 27: 'ʌ',
 28: 'ʒ',
 29: 'ˈ',
 30: '͡',
 31: 'χ'}

In [9]:
# next is for each word, you need to produce the list that has the indexes of its chas

In [12]:
all_verbs_list = all_verbs['Generated Verb-SR'].tolist()

X = [[char_index[char] for char in verb] for verb in all_verbs_list] #predictor variable

In [13]:
from tensorflow.keras.utils import pad_sequences


In [14]:
max_len = max(len(item) for item in X)

In [15]:
X = pad_sequences(X, max_len) #tags zeros to the front

In [16]:
features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
y = all_verbs[features]

In [17]:
X_train, X_test, y_train_pre_encoded, y_test_pre_encoded = train_test_split(X, y, test_size = 0.33, random_state = 1)

In [18]:
feature = 'Aspect'
le = LabelEncoder()
y_train = le.fit_transform(y_train_pre_encoded[feature]) #le.fit() and le.transform() together make le.fit_transform()
y_test = le.transform(y_test_pre_encoded[feature])
print(le.classes_)

['imperfective' 'neutral' 'perfect' 'pluperfect']


In [19]:
lstm_model = tf.keras.models.Sequential()

lstm_model.add(tf.keras.layers.Embedding(input_dim = len(char_index)+1, output_dim = 32, mask_zero = True)) # True ignores zeros # all sequences needs to have the same length but zero willl be ignored
lstm_model.add(tf.keras.layers.LSTM(32)) #can change later
lstm_model.add(tf.keras.layers.Dense(len(le.classes_), activation = 'softmax')) #one output per category

lstm_model.compile(optimizer = 'rmsprop', loss = 'sparse_categorical_crossentropy') #cat cross can have more than two target classes. Sparse : bc we are giving it an integer label

2022-06-08 20:20:11.329168: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
lstm_model.fit(X_train, y_train, epochs = 10) # passes the df 5 times and train on them

Epoch 1/10
924/924 [==============================] - 21s 18ms/step - loss: 0.5934
Epoch 2/10
924/924 [==============================] - 18s 19ms/step - loss: 0.2797
Epoch 3/10
924/924 [==============================] - 16s 18ms/step - loss: 0.2426
Epoch 4/10
924/924 [==============================] - 16s 17ms/step - loss: 0.2249
Epoch 5/10
924/924 [==============================] - 15s 16ms/step - loss: 0.2148
Epoch 6/10
924/924 [==============================] - 16s 17ms/step - loss: 0.2084
Epoch 7/10
924/924 [==============================] - 16s 17ms/step - loss: 0.2035
Epoch 8/10
924/924 [==============================] - 16s 17ms/step - loss: 0.1989
Epoch 9/10
924/924 [==============================] - 16s 17ms/step - loss: 0.1973
Epoch 10/10
924/924 [==============================] - 16s 17ms/step - loss: 0.1947


In [21]:
predictions = lstm_model.predict(X_test) #predicted probability

455/455 [==============================] - 3s 4ms/step


In [22]:
y_pred = np.argmax(predictions, axis = 1) # translates from probabilies to classes

In [23]:
accuracy_score(y_test, y_pred)

0.8698907741979803

In [24]:
confusion_matrix(y_test, y_pred)

array([[3168,    9,    0,    0],
       [  27, 5004,   59,    0],
       [   4,  164, 1551, 1422],
       [   0,    0,  209, 2940]])